# Парсинг текста с картинки

In [1]:
import cv2  # импорт opencv для работы с табличными данными в исходном файле
import pytesseract  # импорт tesseract для оптического распознования текста

Обычное распознование текста через tesseract даёт крайне некачественный результат, в виду наличия линий, что мешают распознованию кириллицы. если провести lagdetect целой таблицы он покажет наличие двух яызков: русского и бельгийского.

In [2]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [3]:
image = cv2.imread('photo_2020-12-11_16-55-23.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

Избавляемся от горизонтальных линий

In [4]:
horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (50,1))
detect_horizontal = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
cnts = cv2.findContours(detect_horizontal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    cv2.drawContours(thresh, [c], -1, (0,0,0), 2)

Избавляемся от вертикальных линий

In [5]:
vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,20))
detect_vertical = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, vertical_kernel, iterations=10)
cnts = cv2.findContours(detect_vertical, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    cv2.drawContours(thresh, [c], -1, (0,0,0), 3)

Расширение текста для удаления точек

In [6]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10,1))
dilate = cv2.dilate(thresh, kernel, iterations=2)
cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    area = cv2.contourArea(c)
    if area < 500:
        cv2.drawContours(dilate, [c], -1, (0,0,0), -1)

In [7]:
result = cv2.bitwise_and(image, image, mask=dilate)
result[dilate==0] = (255,255,255)

In [8]:
data = pytesseract.image_to_string(result, lang='rus',config='--psm 6')
data_lines = data.replace('|', '')
data_mod = data_lines.replace(']', '')
print(data_mod)

0529-1295-06-ТКР5-С

 изм. Кол.уч. Лист  №лок.  Подпись  Дата 
Разработал _  Белина М.В. Стадия  Лист — Листов
Проверил —  никифоров ^ А. 
тИП  Стрижова нле.  Содержание тома 3.5

Никифоров А.А. ОСГиПРи
Н. контр. —  Никифоро  1 РАНС
Гл. спец. —  Дробышев Н.М.

Формат А4



In [9]:
cv2.imshow('thresh', thresh)
cv2.imshow('result', result)
cv2.imshow('dilate', dilate)
cv2.waitKey()
cv2.destroyAllWindows()  # позволяет запустить ячейку на юпитере

In [10]:
text_file = open('Img_parsing.txt', 'w')
wd = text_file.write(data)
text_file.close()

Для распознования цветного штампа OCR не будет недостаточно. Возможно помогла бы нейронная сеть.

# Парсинг текста из документа *pdf

In [ ]:
pip install camelot-py[cv]

Камелот очень неплохо справлется с парсингом текста на кириллице.

In [2]:
import camelot

In [3]:
file = 'РазделПД3.Подр.ПД5_11962.pdf'

Читаем файл

In [4]:
tables = camelot.read_pdf(file, pages = "4, 5", encoding='utf-8')

In [5]:
import pandas as pd

Размер получившийся таблицы:

In [6]:
print(tables[0])

<Table shape=(26, 13)>


Сататистика по парсингу:

In [82]:
tables[0].parsing_report

{'accuracy': 80.01, 'whitespace': 81.36, 'order': 1, 'page': 4}

Из таблицы в датафрейм:

In [7]:
df_p4 = tables[0].df

Нулевая строка включает в себя "2", которая отвечает за номер листа. В данном примере считаю эту часть таблицы не информативной, так как ей соответствует всего одна строка. Поэтому избавляясь от неё есть возможность перенести Обозначение, Наименование, Примечание ближе к заголовку. Также есть проблем с переносом строки, который импортировался как \n, поэтому его заменяю на настоящий перенос строки.

In [8]:
df_p4.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,,,,,,2,,,,,,,
1,,,,,,Обозначение,,,,Наименование,,,Примечание
2,,,,,,0529-1295-06-ТКР5-С,,,,Содержание тома 3.5,,,2
3,,,,,,0529-1295-06-ТКР5-ТЧ,,,,Текстовая часть,,,3
4,,,,,,,,,,Графическая часть,,,


In [9]:
df_p4 = df_p4.drop(0, axis=0)

In [10]:
df_p4.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12
1,,,,,,Обозначение,,,,Наименование,,,Примечание
2,,,,,,0529-1295-06-ТКР5-С,,,,Содержание тома 3.5,,,2
3,,,,,,0529-1295-06-ТКР5-ТЧ,,,,Текстовая часть,,,3
4,,,,,,,,,,Графическая часть,,,
5,,,,,,0529-1295-06-ТКР5-ГЧ1,,,,Перегон Павшино - Тушино \nПлан трассы выноса...,,,29
6,,,,,,0529-1295-06-ТКР5-ГЧ2,,,,Перегон Павшино - Тушино \nПлан трассы выноса...,,,30
7,,,,,,0529-1295-06-ТКР5-ГЧ3,,,,Перегон Павшино - Тушино \nКабельный план вын...,,,31
8,,,,,,0529-1295-06-ТКР5-ГЧ4,,,,Перегон Павшино - Тушино \nПлан трассы кабеле...,,,32
9,,,,,,0529-1295-06-ТКР5-ГЧ5,,,,Перегон Павшино - Тушино \nПлан трассы кабеле...,,,33
10,Согласовано,,,,,,,,,устройств АБ ПК202-ПК199,,,


In [90]:
df_p4 = df_p4.fillna(0)

In [98]:
df_p4[9] = df_p4[9].replace(r'\n', '\n', regex=True)

In [99]:
df_p4.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12
1,,,,,,Обозначение,,,,Наименование,,,Примечание
2,,,,,,0529-1295-06-ТКР5-С,,,,Содержание тома 3.5,,,2
3,,,,,,0529-1295-06-ТКР5-ТЧ,,,,Текстовая часть,,,3
4,,,,,,,,,,Графическая часть,,,
5,,,,,,0529-1295-06-ТКР5-ГЧ1,,,,Перегон Павшино - Тушино План трассы выноса ...,,,29
6,,,,,,0529-1295-06-ТКР5-ГЧ2,,,,Перегон Павшино - Тушино План трассы выноса ...,,,30
7,,,,,,0529-1295-06-ТКР5-ГЧ3,,,,Перегон Павшино - Тушино Кабельный план выно...,,,31
8,,,,,,0529-1295-06-ТКР5-ГЧ4,,,,Перегон Павшино - Тушино План трассы кабелей...,,,32
9,,,,,,0529-1295-06-ТКР5-ГЧ5,,,,Перегон Павшино - Тушино План трассы кабелей...,,,33
10,Согласовано,,,,,,,,,устройств АБ ПК202-ПК199,,,


In [105]:
df_p4[11] = df_p4[11].replace(r'\n', ' ', regex=True)

In [106]:
df_p4

,0,1,2,3,4,5,6,7,8,9,10,11,12
1,,,,,,Обозначение,,,,Наименование,,,Примечание
2,,,,,,0529-1295-06-ТКР5-С,,,,Содержание тома 3.5,,,2
3,,,,,,0529-1295-06-ТКР5-ТЧ,,,,Текстовая часть,,,3
4,,,,,,,,,,Графическая часть,,,
5,,,,,,0529-1295-06-ТКР5-ГЧ1,,,,Перегон Павшино - Тушино План трассы выноса ...,,,29
6,,,,,,0529-1295-06-ТКР5-ГЧ2,,,,Перегон Павшино - Тушино План трассы выноса ...,,,30
7,,,,,,0529-1295-06-ТКР5-ГЧ3,,,,Перегон Павшино - Тушино Кабельный план выно...,,,31
8,,,,,,0529-1295-06-ТКР5-ГЧ4,,,,Перегон Павшино - Тушино План трассы кабелей...,,,32
9,,,,,,0529-1295-06-ТКР5-ГЧ5,,,,Перегон Павшино - Тушино План трассы кабелей...,,,33
10,Согласовано,,,,,,,,,устройств АБ ПК202-ПК199,,,


In [133]:
from PyPDF2 import PdfFileReader, PdfFileWriter

In [140]:
inputpdf = PdfFileReader(open("РазделПД3.Подр.ПД5_11962.pdf", "rb"))

In [135]:
file_base_name = pdf_file_path.replace('.pdf', '')

Чтобы взять определённую страницу использован цикл, который при помощи pyPDF2 делит искомый файл на совокупность файлов формата pdf

In [141]:
for i in range(inputpdf.numPages):
    output = PdfFileWriter()
    output.addPage(inputpdf.getPage(i))
    with open("document-page%s.pdf" % i, "wb") as outputStream:
        output.write(outputStream)

In [157]:
from tika import parser

In [158]:
raw = parser.from_file('document-page4.pdf', xmlContent=True)

In [166]:
print(raw['content'])

<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta name="pdf:PDFVersion" content="1.3" />
<meta name="X-Parsed-By" content="org.apache.tika.parser.DefaultParser" />
<meta name="X-Parsed-By" content="org.apache.tika.parser.pdf.PDFParser" />
<meta name="pdf:hasXFA" content="false" />
<meta name="access_permission:modify_annotations" content="true" />
<meta name="access_permission:can_print_degraded" content="true" />
<meta name="access_permission:extract_for_accessibility" content="true" />
<meta name="access_permission:assemble_document" content="true" />
<meta name="xmpTPg:NPages" content="1" />
<meta name="resourceName" content="b'document-page4.pdf'" />
<meta name="pdf:hasXMP" content="false" />
<meta name="dc:format" content="application/pdf; version=1.3" />
<meta name="access_permission:extract_content" content="true" />
<meta name="access_permission:can_print" content="true" />
<meta name="access_permission:fill_in_form" content="true" />
<meta name="pdf:encrypted" content="

Создаём строку на основе полученных данных. Разделяем строки методом strip:

In [203]:
transform = list()
i = 1
new_cont = raw['content'].splitlines()
for line in new_cont:
    transform.append(line)
    i =+ 1

In [217]:
df_cont = pd.DataFrame([transform])

In [214]:
df_cont

,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,"<html xmlns=""http://www.w3.org/1999/xhtml"">",<head>,"<meta name=""pdf:PDFVersion"" content=""1.3"" />","<meta name=""X-Parsed-By"" content=""org.apache.t...","<meta name=""X-Parsed-By"" content=""org.apache.t...","<meta name=""pdf:hasXFA"" content=""false"" />","<meta name=""access_permission:modify_annotatio...","<meta name=""access_permission:can_print_degrad...","<meta name=""access_permission:extract_for_acce...","<meta name=""access_permission:assemble_documen...",...,ано,</p>,<p>,,</p>,<p>1 Лист П Стадия 0529-1295-06-ТКР5-ТЧ Н. кон...,Коноваленко Е.Н. Изм. Кол.уч. Лист...,<p />,</div>,</body></html>
